In [167]:
import numpy
import sys
from tqdm import tqdm
import re
import pickle
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import LSTM, Embedding, Input
from keras.layers import Dropout
from keras.layers import Dense

Данные - выборка из 1млн русских анекдотов

In [58]:
raw_text = open("/srv/kkotochigov/JokeGenerator/data/jokes").read()

Переводим в нижний регистр, оставляем русские буквы, цифры и некоторую пунктуацию. Так как данные замусоренные, это сокращает словарь с 200 до до 50 символов.

In [59]:
raw_text = raw_text.replace("\\\n"," ").replace("\n###","\n").lower().replace('"','').replace('\\','')
raw_text = re.sub('[^\-0-9_а-я:,.\\!\\?\\n]', ' ', raw_text).replace('  ',' ').replace('   ',' ').replace('    ',' ')

### Делаем словарь символов для кодировки

In [60]:
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
int_to_char = dict((i, c) for i, c in enumerate(chars))
with open("/srv/kkotochigov/char_mapper.pickle","wb") as file_handle:
        pickle.dump((char_to_int, int_to_char), file_handle)
n_chars = len(raw_text)
n_vocab = len(chars)

In [ ]:
seq_length = 30
dataX=[]
dataY=[]

In [ ]:
# with open("/srv/kkotochigov/char_mapper.pickle","r") as file_handle:
#     char_to_int1, int_to_char1 = pickle.load(file_hadle)

# n_chars = len(raw_text)
# n_vocab = len(chars)

# seq_length = 30
# dataX=[]
# dataY=[]

Нарезаем текст на кейсы скользящим окном (размером seq_length)

Пример такого кейса [125,0,7,15,2] => 250

In [61]:
for line in tqdm(raw_text.split("\n"), total=len(raw_text.split("\n")), unit=" strings"):
    full_line = line + '\n'
    for i in range(0,len(full_line) - seq_length):
        seq_in = full_line[i:i+seq_length]
        seq_out = full_line[i+seq_length]
        dataX.append([char_to_int[c] for c in seq_in])
        dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print ("Total patterns:" + str(n_patterns))

100%|██████████| 192591/192591 [01:10<00:00, 2744.21 strings/s]

Total patterns:18225940


Нормализуем все закодированные символы

In [62]:
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
X = X / float(n_vocab)
y = np_utils.to_categorical(dataY)

Создаем сеть. Изначальная Архитектура - два LSTM слоя (256) и один Dense(256 -> 1)

In [63]:
model = Sequential()
model.add(LSTM(256,input_shape=(X.shape[1],X.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation="softmax"))
model.compile(loss="categorical_crossentropy", optimizer="adam")

Model with Embedding

In [56]:
model = Sequential()
model.add(Input(shape=(30,)))
model.add(Embedding(input_dim = n_chars, output_dim = 256, input_length=30))
model.add(LSTM(256,return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(Dense(16, activation="relu"))
model.add(Dense(y.shape[1], activation="softmax"))
model.compile(loss="categorical_crossentropy", optimizer="adam")

TypeError: The added layer must be an instance of class Layer. Found: Tensor("input_1:0", shape=(?, 30), dtype=float32)

Для тестирования нужен сэмпл, иначе на 19 млн кейсов долго будут считаться

In [64]:
sample_size = 10000000
import random
import pandas
import datetime
sample_indices = random.sample(range(X.shape[0]),sample_size)
X_sample = X[sample_indices, :]
y_sample = y[sample_indices]

Создадим несколько обработчиков.
- CSVLogger - чтобы после закрытия было видно, что он делает
- lambdaCallback - с выводом времени старта
- modelCheckpoint - чтобы потом можно было подгрузить модель, если kernel упадет

In [65]:
# Define Callbacks
from keras.callbacks import LambdaCallback, CSVLogger, ModelCheckpoint
lambdaCallback = LambdaCallback(on_epoch_begin = lambda epoch, logs: print("Epoch = {}, TS = {}".format(epoch, datetime.datetime.now())))
csv_logger = CSVLogger('/srv/aprosvetov/training.log')
model_checkpoint = ModelCheckpoint('/srv/aprosvetov/model.h5')

Так как эпохи выводятся в лог и на диск, ставлю verbose = 0. Если оставить verbose = 1, браузер начинает виснуть.
С batch_size больше 4000 не работает.

In [138]:
model.fit(X_sample, y_sample, epochs=1, batch_size=1024, verbose=0, callbacks = [lambdaCallback, csv_logger, model_checkpoint])

Epoch = 0, TS = 2019-05-08 16:11:11.615249


Обучаемую Модель (50 эпох, 30 символов) созхдранил 
/srv/aprosvetov/model.h5

В свою директорию прав нету из-под пользователя aprosvetov

Сохранить модель

In [137]:
model.save("/srv/kkotochigov/JokeGenerator/model.h5")

OSError: Unable to create file (unable to open file: name = '/srv/kkotochigov/JokeGenerator/model.h5', errno = 13, error message = 'Permission denied', flags = 13, o_flags = 242)

In [139]:
use_predefined_model = True
if use_predefined_model:
    with open("/srv/kkotochigov/char_mapper.pickle","rb") as file_handle:
        char_to_int, int_to_char = pickle.load(file_handle)
n_vocab = len(char_to_int)

NameError: name 'pickle' is not defined

In [ ]:
# model.fit(X, y, epochs=50, batch_size=1024)

Попробуем генерацию текста. Берем случайное начало текста, генерируем наиболее вероятный символ и добавляем его в результат (seq_in).

Начальный текст должен быть ровно 100 символов маленькиими буквами.

Генерируемый символ выбираем не жадно, как в оригинале, а из двух наиболее веротяных

In [114]:
def create_pattern(pattern):
    return [char_to_int[x] for x in pattern.lower()[0:30]]

def weighted_choice(choices):
   total = sum(w for c, w in choices)
   r = random.uniform(0, total)
   upto = 0
   for c, w in choices:
      if upto + w >= r:
         return c
      upto += w
   assert False, "Shouldn't get here"

def generate(pattern, top_n=1):
    i = 0
    result = ""
    result_text = ''
    while (result != "\n") and (i < 300):
        x = numpy.reshape(pattern, (1, len(pattern), 1))
        x = x / float(n_vocab)
        prediction = model.predict(x, verbose=0)
        if top_n == 1:
            index = numpy.argmax(prediction)
        else:
            choices = sorted(list(zip(range(0,len(prediction[0])), prediction[0])), key=lambda x: -x[1])[0:top_n]
            index = weighted_choice(choices)
        result = int_to_char[index]
        seq_in = [int_to_char[value] for value in pattern]
        result_text += result
        pattern.append(index)
        pattern = pattern[1:len(pattern)]
        i = i + 1
    return result_text

def write_pattern(pattern):
    return "".join([int_to_char[x] for x in pattern])

In [165]:
pattern = "Чем больше женщину мы любим, т"[-30:]
pattern_int = create_pattern(pattern)
print("***")
print("%s%s" % (pattern, generate(pattern_int, top_n=2)))

***
Чем больше женщину мы любим, так вот вы подошел с подругамом в карман и подошел с продавецами и послотривает в свою себе в стороны и старушки не просит стоит в своей доме и спать... 



In [ ]:
prediction_model = load("/srv/aprosvetov/model.h5")

In [36]:
model.save("/srv/kkotochigov/model.h5")

{0: '\n',
 1: ' ',
 2: '!',
 3: ',',
 4: '-',
 5: '.',
 6: '0',
 7: '1',
 8: '2',
 9: '3',
 10: '4',
 11: '5',
 12: '6',
 13: '7',
 14: '8',
 15: '9',
 16: ':',
 17: '?',
 18: '_',
 19: 'а',
 20: 'б',
 21: 'в',
 22: 'г',
 23: 'д',
 24: 'е',
 25: 'ж',
 26: 'з',
 27: 'и',
 28: 'й',
 29: 'к',
 30: 'л',
 31: 'м',
 32: 'н',
 33: 'о',
 34: 'п',
 35: 'р',
 36: 'с',
 37: 'т',
 38: 'у',
 39: 'ф',
 40: 'х',
 41: 'ц',
 42: 'ч',
 43: 'ш',
 44: 'щ',
 45: 'ъ',
 46: 'ы',
 47: 'ь',
 48: 'э',
 49: 'ю',
 50: 'я'}

In [65]:
"Приходит Чукча к жене и говорит: - Как же у тебя тут Вальдшнепы снуют? - Да не боись ты, они же ручные. ".lower()[0:100]

'приходит чукча к жене и говорит: - как же у тебя тут вальдшнепы снуют? - да не боись ты, они же ручн'

### Несколько примеров того, что получается (1млн, 50 эпох)

лучили приворили сосос по саком протто подошел и подходит к место:- нет, там вот в принере самого собирается в карме. после себя подоравляет самомату проверими подоворите.. ну, вот, не саком сако и сорашивает машину и сам возмущиноо проверил в сортоянееми и саком посом соррил в самомете, поставились и саког соросили в саком соварещ. подаегает к себе сак и спрашивает: - чего вы так волит??? нет, не понимаете, через положение он сам подходит к ней. - не понимаете, а ты подомой, а ты что вы на собой подовооете? - ну почему? - а вот в поле на себя прикосится на себя в постели. послошрет в коннату и говорит: - дорогой, тогда ято только что вы просто сак надо было подовоомись и сорашиние в постря в пострянени под соседним сорова соседа не понял, что ты придуать, что вы посом приходит в соседний саком делать начинает в карты и сак посом приходил в самомете.  подходит к мамьчику и говорит:  - ну, через некоторое время, на следующей день вовочка:  - да вот в картане подоворились.  - а что ты пр

идется высотки и на себя в полоте последнее советского сортоб состива  - доктор, вы послотрел на саком соседе и сорашивает:  - а чего так вы не просто продолжили под состобными.  - а ты че ты не понять?  - да вот, на следующий день в какой постоя текель на солнаеенных простого полазать в самомет и придложит в состива... после теко не выделж в картане и сакого просто послолр просила после таког не привет, а потом сорашивают комес и сам на середь придласает по приворовини и сорадает и придласил просто последнее сак просит сорашивает: - что, вот только привотовили подоворите. просто придется на первый раз в состир. - не забора на полу и выпить. - а что вы подуматите проводить не полазывался? - а вот то ведь не подова, а вы не сорашиваете самое подоврожнения водитель на саботу пассажиры? - ну что же ты придомались? - нет, не понимаешь, на собой под дрргой подоврегается. - нет. не подовратим. посом просто сорроятносесте себе сам подоидать собирает подоможник. - ну что, на слочей вы не собет


### длина = 30 символов
приходит чукча к жене и говорит:- соади, конода придет на себя солько володу.  - на предртражини машение постоятно сак соращивает: - ну, что ты сеелать ведь не подертовали..

так и что сказали вам, что если вы 
и вы подходите?  - ну, когда я воду не пришел сакой маланьким. 
***

### На 10 млн обучено 36 эпох
***
Приходит муж в магазин и говор ит после состоит с собакой.  
А что такое только получилось?  
Ну тогда после него просит спрашивать, что вы просто спрашивают, что она последних стороны.  
Ну, ты чего тогда в картошку? 
Да, понимаешь, когда вы пришли в старуи получилости и спросили, что она не положела.  
Не знаю, что в старо
***

### 50 эпох
***


Приходит муж в магазин и говорит придет с надеждой: - ну что, не подумала меня...

Приходит муж в магазин и говорит: - дорогой, ты что, такое волку представляет вовы водила с проблемами.

Приходит муж в магазин и говорит: - да вот вы придумали старый домой и стоит в столовой под колени и поднимает в старой машину и говорит:  - ну, ты сколько меня не пришла? - да, но не волнуюшь на себя в кармане возникии в столовой под колнатом.

***
Возвращается муж из командировкя с половом мужийом проститутками. 

***
Входит Чукча в юрту и говорит просто и продал с соседний дом и приходит с собачками и подходит к себе в себе в постель...



In [ ]:
import numpy
import pickle
import random
from keras.models import load_model

model = load_model("/srv/kkotochigov/model.h5")

use_predefined_model = True
if use_predefined_model:
    with open("/srv/kkotochigov/char_mapper.pickle","rb") as file_handle:
        char_to_int, int_to_char = pickle.load(file_handle)

n_vocab = len(char_to_int)

def create_pattern(pattern):
    return [char_to_int[x] for x in pattern.lower()[0:30]]

def weighted_choice(choices):
   total = sum(w for c, w in choices)
   r = random.uniform(0, total)
   upto = 0
   for c, w in choices:
      if upto + w >= r:
         return c
      upto += w
   assert False, "Shouldn't get here"

def generate(pattern, top_n=1):
    i = 0
    result = ""
    result_text = ''
    while (result != "\n") and (i < 300):
        x = numpy.reshape(pattern, (1, len(pattern), 1))
        x = x / float(n_vocab)
        prediction = model.predict(x, verbose=0)
        if top_n == 1:
            index = numpy.argmax(prediction)
        else:
            choices = sorted(list(zip(range(0,len(prediction[0])), prediction[0])), key=lambda x: -x[1])[0:top_n]
            index = weighted_choice(choices)
        result = int_to_char[index]
        seq_in = [int_to_char[value] for value in pattern]
        result_text += result
        pattern.append(index)
        pattern = pattern[1:len(pattern)]
        i = i + 1
    return result_text

def write_pattern(pattern):
    return "".join([int_to_char[x] for x in pattern])

pattern = "Чем больше женщину мы любим, т"[-30:]
pattern_int = create_pattern(pattern)
print("***")
print("%s%s" % (pattern, generate(pattern_int, top_n=3)))

In [ ]:
from flask import Flask
app = Flask(__name__)
from keras.models import load_model
import random
import numpy
import pickle

def @app.route("/generate/<start_text>"):

    model = load_model("/srv/kkotochigov/model.h5")
    use_predefined_model = True
    if use_predefined_model:
       with open("/srv/kkotochigov/char_mapper.pickle","rb") as file_handle:
         char_to_int, int_to_char = pickle.load(file_handle)
    n_vocab = len(char_to_int)

    def create_pattern(pattern):
        return [char_to_int[x] for x in pattern.lower()[0:30]]

    def weighted_choice(choices):
       total = sum(w for c, w in choices)
       r = random.uniform(0, total)
       upto = 0
       for c, w in choices:
          if upto + w >= r:
             return c
          upto += w
       assert False, "Shouldn't get here"

    def generate_text(pattern, top_n=1):
        i = 0
        result = ""
        result_text = ''
        while (result != "\n") and (i < 300):
            x = numpy.reshape(pattern, (1, len(pattern), 1))
            x = x / float(n_vocab)
            prediction = model.predict(x, verbose=0)
            if top_n == 1:
                index = numpy.argmax(prediction)
            else:
                choices = sorted(list(zip(range(0,len(prediction[0])), prediction[0])), key=lambda x: -x[1])[0:top_n]
                index = weighted_choice(choices)
            result = int_to_char[index]
            seq_in = [int_to_char[value] for value in pattern]
            result_text += result
            pattern.append(index)
            pattern = pattern[1:len(pattern)]
            i = i + 1
        return result_text

    def write_pattern(pattern):
        return "".join([int_to_char[x] for x in pattern])

    start_text = "Приходит муж домой и говорит жене"
    pattern = start_text[-30:]
    pattern_int = create_pattern(pattern)
    generated = "%s%s" % (pattern, generate_text(pattern_int, top_n=3))

    return generated
